In [1]:
magma_dir = '/home/ubuntu/magma/'
bucket_dir = '/home/ubuntu/s3/'
transformers_dir = '/home/ubuntu/transformers/'
cache_dir = bucket_dir+'.cache/'

## **Fine-tuning**

In [2]:
finetune_script = '"'+transformers_dir+'examples/seq2seq/finetune_trainer.py"'
eval_script = '"'+transformers_dir+'examples/seq2seq/run_eval.py"'

### **Config**

In [3]:
import sys
sys.path.insert(0, magma_dir)
import config

import wandb
wandb.login()

project_name = 'hps_t5_para_wordembed'
%env WANDB_PROJECT=$project_name

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: marcoabrate (use `wandb login --relogin` to force relogin)


env: WANDB_PROJECT=hps_bart_para_wordembed


### HP search

In [4]:
model_name_or_path = 't5-large'

In [5]:
data_dir = '"'+bucket_dir+'datasets/karger_books_para_wordembed/t5/st/"'

output_dir = '"'+bucket_dir+'fine-tuning/'+project_name+'"'

log_dir = output_dir + '/logs'

In [6]:
hp_search_checkpoint_dir = output_dir + '/ray_checkpoints'

hp_search_log_dir = output_dir + '/ray_results'

In [7]:
from transformers import AutoConfig
model_config = AutoConfig.from_pretrained(model_name_or_path, use_cache=False)
model_config.min_length = config.ONE_BULLET_MIN_LEN
model_config.max_length = config.ONE_BULLET_MAX_LEN
model_config.num_beams = 2

model_config_dir = '"'+bucket_dir+'fine-tuning/'+\
    model_name_or_path.replace('/', '?')+'_config"'
model_config.save_pretrained(model_config_dir[1:-1])

In [7]:
!python3 $finetune_script \
--model_name_or_path $model_name_or_path \
--config_name $model_config_dir \
--tokenizer_name $model_name_or_path \
--cache_dir $cache_dir \
--data_dir $data_dir \
--freeze_embeds \
--hp_search \
--hp_search_checkpoint_dir $hp_search_checkpoint_dir --hp_search_db_host '0.0.0.0' \
--hp_search_name $project_name --hp_search_log_dir $hp_search_log_dir \
--label_smoothing_factor 0.1 \
--warmup_steps 0 \
--fp16 \
--sortish_sampler \
--task summarization \
--max_source_length 512 \
--max_target_length $config.ONE_BULLET_MAX_LEN \
--val_max_target_length $config.ONE_BULLET_MAX_LEN \
--num_train_epochs 10 \
--logging_steps 20 --logging_first_step \
--per_device_train_batch_size 2 --per_device_eval_batch_size 8 \
--evaluation_strategy steps --eval_beams 2 \
--predict_with_generate \
--save_steps 5000 \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $config.SEED \
--run_name $output_dir

8 40
16 20
32 10
